- Calificación de IMDB.
- Dirección (director o directores).
- Guionistas.
- Argumento.
- Duración (en minutos).

In [27]:
import pandas as pd
from tqdm import tqdm

import pickle 
import sys
sys.path.append('../')

In [28]:
from bs4 import BeautifulSoup
import requests

In [29]:
with open(f'../data/full_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [30]:
df = pd.DataFrame(data, columns=['type', 'title', 'year', 'month', 'id', 'genre'])

In [31]:
df['year'].unique()

array([2024.,   nan, 2025., 2023., 2022.])

Vemos que realmente solo tenemos información de películas de 2022 a 2025. Esto indica que tendríamos que revisar que la extracción de información de la API funciona correctamente en el paso anterior.

De momento vamos a continuar con esta información, y si nos sobra tiempo lo hacemos.

In [32]:
df.duplicated().sum()

0

No hay duplicados, así que bien. Pero...

In [33]:
df['id'].duplicated().sum()

1115

Resulta que hay duplicados porque algunas películas tienen varios géneros. Por el momento lo vamos a dejar así y usaremos el índice como identificador de la entrada

In [34]:
df[df['id'].isin(df[df['id'].duplicated()]['id'].unique())].sort_values(by='id').head(10)

,type,title,year,month,id,genre
0,movie,Mom's Coming,2024.0,11.0,tt10579986,Drama
4000,movie,Mom's Coming,2024.0,11.0,tt10579986,Horror
7000,movie,Mom's Coming,2024.0,11.0,tt10579986,Thriller
2000,movie,Slasher House 3,2024.0,3.0,tt11078340,Action
4001,movie,Slasher House 3,2024.0,3.0,tt11078340,Horror
3000,movie,Amigos imaginarios,2024.0,5.0,tt11152168,Fantasy
1001,movie,Amigos imaginarios,2024.0,5.0,tt11152168,Comedy
3,movie,Amigos imaginarios,2024.0,5.0,tt11152168,Drama
5214,movie,Vincent,2023.0,NaN,tt11253182,Mystery
3220,movie,Vincent,2023.0,NaN,tt11253182,Fantasy


Para que sea viable un scrapeo en poco tiempo vamos a utilizar solamente algunas películas. 

Comencemos utilizando solo las de 2024 que sean drama y no sean cortos, solo largometrajes.

In [35]:
filtro = (df['year'] == 2024) & (df['genre'] == 'Drama') & (df['type'] == 'movie')

df_filtrado = df[filtro]

df_filtrado.shape[0]

489

De hecho comprobamos que no haya duplicados ahora.

In [36]:
df_filtrado['id'].duplicated().sum()

0

Tenemos 489 películas, vayamos con esto por el momento

In [37]:
def get_synopsis(id_):

    url = f'https://www.imdb.com/title/{id_}/plotsummary'

    # Simulamos un usuario
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)

    # Comprobamos que obtenemos un 200
    if response.status_code != 200:
        print("Error:", response.status_code)
        print("id:", id_)
        return

    soup = BeautifulSoup(response.content, 'html.parser')

    sinopsis = soup.find("div", {"data-testid": "sub-section-synopsis"}).text

    return sinopsis

In [38]:
def scrap_id(id_):

    url = f'https://www.imdb.com/es-es/title/{id_}/'

    # Simulamos un usuario
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    response = requests.get(url, headers=headers)

    # Comprobamos que obtenemos un 200
    if response.status_code != 200:
        print("Error:", response.status_code)
        print("id:", id_)
        return

    # Let him cook (the soup)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Puntuación
    try:
        score_ = soup.find("div", {'data-testid': "hero-rating-bar__aggregate-rating__score"}).text
        score_ = float(score_.split('/')[0].replace(',', '.'))
    except:
        score_ = None

    # Dirección, Guión, Reparto Principal
    info = soup.find("ul", {'class':"ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt"})
    info_table = info.findAll("li", {"class": "ipc-inline-list__item"})

    directores_ = []
    guionistas_ = []
    actores_ = []

    for item in info_table:

        class_ = item.a.get('href').split('=')[1]

        if 'dr' in class_:
            directores_.append(item.text)

        elif 'wr' in class_:
            guionistas_.append(item.text)

        elif 'st' in class_:
            actores_.append(item.text)

    # Resumen
    try:
        resumen_ = soup.find("span", {'data-testid':"plot-l"}).text
    except:
        resumen_ = None

    # Duración
    try:
        duracion_ = soup.find("ul", {"class":"ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt"}).findAll("li", {'class': "ipc-inline-list__item"})[-1].text
        # Si dura menos de 1h no aparece h
        if 'h' in duracion_:
            horas, minutos = duracion_.split(' ')
            duracion_ = int(horas.rstrip('h').strip())*60 + int(minutos.rstrip('min').strip())
        else:
            duracion_ = int(minutos.rstrip('min').strip())


    except:
        duracion_ = None

    # Sinopsis
    try:
        sinopsis_ = get_synopsis(id_)

    except:
        sinopsis_ = None

    return (id_, score_, directores_, guionistas_, actores_, resumen_, sinopsis_, duracion_)

In [39]:
datos = []

for id_ in tqdm(df_filtrado['id'].values):
    datos.append(scrap_id(id_))

  3%|▎         | 15/489 [00:42<19:41,  2.49s/it]

Error: 404


  4%|▍         | 20/489 [00:58<21:32,  2.76s/it]

Error: 404


  4%|▍         | 22/489 [01:03<18:02,  2.32s/it]

...

Error: 404


 15%|█▌        | 74/489 [03:13<14:30,  2.10s/it]

Error: 404


 15%|█▌        | 74/489 [03:13<18:05,  2.62s/it]


KeyboardInterrupt: 

In [16]:
pd.DataFrame(datos, columns=['id', 'score', 'directores', 'guionistas', 'actores', 'resumen', 'sinopsis', 'duracion'])

,id,score,directores,guionistas,actores,resumen,sinopsis,duracion
0,tt10597228,None,"[Erik Krefeld, Eddel Martinez]","[Stevie Kinchen, Erik Krefeld]","[Aaron Jackson, Rose LaRue, Cassidy Reyes]",Esta es una historia sobre encontrar el valor ...,None,90.0
1,tt13561128,None,[Geoffrey Pheasant],[Geoffrey Pheasant],"[Tao Guo, Betty Faye, Aitana Llorente]",Three separate couples each decide to make som...,Three Couples Freed's poetic cinematic vision ...,85.0
2,tt13818368,"6,2/10",[Siddharth Anand],"[Siddharth Anand, Ramon Chibb, Abbas Dalal]","[Hrithik Roshan, Deepika Padukone, Anil Kapoor]",Un joven con aspiraciones llamado Shamsher Pat...,Maqsood Abrar (Mushtaq Kak) is a Jihadi from J...,166.0
3,tt14564000,"5,4/10",[Trivikram Srinivas],[Trivikram Srinivas],"[Mahesh Babu, Sreeleela, Meenaakshi Chaudhary]","Sigue a Guntur Karam, el rey del inframundo de...",None,159.0
4,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...
262,tt30825258,"9,0/10","[Lyubomir Petkov, Tsvetoslav Tsonev]","[Lyubomir Petkov, Tsvetoslav Tsonev]","[Maya Tinkova, Vanessa Peyankova, Katia Kazakova]","Trying to protect her younger sisters, Svetlan...",None,NaN
263,tt30825695,None,[Eric Fankhauser],[],"[Marisa Alu, Eric Fankhauser, Edna Hirsbrunner]",The plot resembles the directors personal rela...,None,NaN
264,tt30825793,None,[Aakash Tyagi],[],"[Mohammed Naushad, Shivani Yadav, Sarita Chatu...",,None,NaN
265,tt30827186,None,[Sean Ryan],[Sean Ryan],"[Sabrina Orro, Sean Ryan]",When a blow out fight takes an unexpected turn...,None,NaN
